# library

In [1]:
import cv2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import sys
import numpy as np

/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/evan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

Instructions for updating:
non-resource variables are not supported in the long term


# Character Table

In [2]:
character = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
              'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '川', '鄂', '赣', '甘', '贵',
              '桂', '黑', '沪', '冀', '津', '京', '吉', '辽', '鲁', '蒙', '闽', '宁', '青', '琼', '陕', '苏', '晋',
              '皖', '湘', '新', '豫', '渝', '粤', '云', '藏', '浙']

# Function

In [3]:
# This funciton are usd to remove the small area of the image. the tiny scale would be delete.
def vscale(rotate_rect):
   threthold = 0.4
   aspect = 4#4.7272
   MIN = 10*(10*aspect)
   MAX = 150*(150*aspect)
   min_aspect = aspect*(1-threthold)
   max_aspect = aspect*(1+threthold)
   theta = 30
   if rotate_rect[1][0]==0 or rotate_rect[1][1]==0:
       return False
   r = rotate_rect[1][0]/rotate_rect[1][1]
   r = max(r,1/r)
   area = rotate_rect[1][0]*rotate_rect[1][1]
   if area>MIN and area<MAX and r>min_aspect and r<max_aspect:
         # the angle of the rectangular can not over 4/pi
       if ((rotate_rect[1][0] < rotate_rect[1][1] and rotate_rect[2] >= -90 and rotate_rect[2] < -(90 - theta)) or
               (rotate_rect[1][1] < rotate_rect[1][0] and rotate_rect[2] > -theta and rotate_rect[2] <= 0)):
           return True
   return False

# To adjust the angle of image or the character angle of the image to avoid the image was shooting by strange angle
# If the image was shooting in strange angle, license plate would not parallel to the edge of the image
# And the character would have different size, shape or height compare with normal condition.
# So this function is used to transform the license plate part looks like normal condition
def image_Transform(car, img):
    return_flag = False
    has_throeed = True
    height,weidth = img.shape[:2]
    wedith_react,heigth_react = car[1][0], car[1][1]
    angle = car[2]
    if car[2]==0:
        return_flag = True
        has_throeed = False
    if car[2]==-90 and wedith_react<heigth_react:
        wedith_react, heigth_react = heigth_react, wedith_react
        return_flag = True
        has_throeed = False
    if return_flag:
        car_img = img[int(car[0][1] - heigth_react / 2):int(car[0][1] + heigth_react / 2),
                  int(car[0][0] - wedith_react / 2):int(car[0][0] + wedith_react / 2)]
        return car_img
    car = (car[0], (wedith_react, heigth_react), angle)
    box = cv2.boxPoints(car)
    height_p = right_point = [0,0]
    left_p = low_point = [car[0][0], car[0][1]]
    for p in box:
        if left_p[0] > p[0]:
            left_p = p
        if low_point[1] > p[1]:
            low_point = p
        if height_p[1] < p[1]:
            height_p = p
        if right_point[0] < p[0]:
            right_point = p
    if left_p[1] <= right_point[1]:
        new_right_point = [right_point[0], height_p[1]]
        pts1 = np.float32([left_p, height_p, right_point])
        pts2 = np.float32([left_p, height_p, new_right_point])
        M = cv2.getAffineTransform(pts1, pts2)
        dst = cv2.warpAffine(img, M, (round(weidth * 2), round(height * 2)))
        car_img = dst[int(left_p[1]):int(height_p[1]), int(left_p[0]):int(new_right_point[0])]
    elif left_p[1] > right_point[1]:
        new_left_point = [left_p[0], height_p[1]]
        pts1 = np.float32([left_p, height_p, right_point])
        pts2 = np.float32([new_left_point, height_p, right_point])
        M = cv2.getAffineTransform(pts1, pts2)
        dst = cv2.warpAffine(img, M, (round(weidth * 2), round(height * 2)))
        car_img = dst[int(right_point[1]):int(height_p[1]), int(new_left_point[0]):int(right_point[0])]

    return car_img


# Some times CV2.imshow would cause error.
def preprocessing(orig):
    gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    blur = cv2.blur(gray, (3, 3))
    sobel = cv2.Sobel(blur, cv2.CV_16S, 1, 0, ksize=3)
    sobel = cv2.convertScaleAbs(sobel)
    hsv = cv2.cvtColor(orig, cv2.COLOR_BGR2HSV)
    h, s, v = hsv[:, :, 0], hsv[:, :, 1], hsv[:, :, 2]
    # Yellow between[26,34] blue[100,124]
    # Our project default in blue.
    blue = (((h > 26) & (h < 34)) | ((h > 100) & (h < 124))) & (s > 70) & (v > 70)
    blue = blue.astype('float32')
    mix = np.multiply(sobel, blue)
    mix = mix.astype(np.uint8)
    ret, binary = cv2.threshold(mix, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    kk = cv2.getStructuringElement(cv2.MORPH_RECT,(21,5))
    close = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kk)
    return close

# verify it was reel lincense palte by flood fill algorithm.
def verify_color(rotate_rect,src_image):
    height,wedith = src_image.shape[:2]
    mask_point = np.zeros(shape=[height+2,wedith+2],dtype=np.uint8)
    kernal2 = np.ones((5, 27), np.float64)
    kernel = np.ones((5, 23), np.uint8)
    connectivity = 4
    lower,upper = 30,30
    new_value = 255
    flags = connectivity
    flags |= cv2.FLOODFILL_FIXED_RANGE  # consider the different betweeen seed and pixel
    flags |= new_value << 8
    flags |= cv2.FLOODFILL_MASK_ONLY #mask the part of image that not the lincens plate.
    rand_seed_num = 5000 #generate random seed.
    valid_seed_num = 200 #Select 200 vaild seed
    newp = 0.1
    points_select = cv2.boxPoints(rotate_rect)
    points_x = [n[0] for n in points_select]
    points_x.sort(reverse=False)
    adjust_x = int((points_x[2]-points_x[1])*newp)
    range_col = [points_x[1]+adjust_x,points_x[2]-adjust_x]
    points_y = [n[1] for n in points_select]
    points_y.sort(reverse=False)
    new_y = int((points_y[2]-points_y[1])*newp)
    row_range = [points_y[1]+new_y, points_y[2]-new_y]
    if (range_col[1]-range_col[0])/(points_x[3]-points_x[0])<0.4\
        or (row_range[1]-row_range[0])/(points_y[3]-points_y[0])<0.4:
        row_of_points = []
        col_of_points = []
        for i in range(2):
            pt1,pt2 = points_select[i],points_select[i+2]
            x_adjust,y_adjust = int(newp*(abs(pt1[0]-pt2[0]))),int(newp*(abs(pt1[1]-pt2[1])))
            if (pt1[0] <= pt2[0]):
                pt1[0], pt2[0] = pt1[0] + x_adjust, pt2[0] - x_adjust
            else:
                pt1[0], pt2[0] = pt1[0] - x_adjust, pt2[0] + x_adjust
            if (pt1[1] <= pt2[1]):
                pt1[1], pt2[1] = pt1[1] + new_y, pt2[1] - new_y
            else:
                pt1[1], pt2[1] = pt1[1] - y_adjust, pt2[1] + y_adjust
            x_list = [int(x) for x in np.linspace(pt1[0],pt2[0],int(rand_seed_num /2))]
            list_y = [int(y) for y in np.linspace(pt1[1],pt2[1],int(rand_seed_num /2))]
            col_of_points.extend(x_list)
            row_of_points.extend(list_y)
    else:
        row_of_points = np.random.randint(row_range[0],row_range[1],size=rand_seed_num)
        col_of_points = np.linspace(range_col[0],range_col[1],num=rand_seed_num).astype(np.int)
    row_of_points = np.array(row_of_points)
    col_of_points = np.array(col_of_points)
    hsv_img = cv2.cvtColor(src_image, cv2.COLOR_BGR2HSV)
    h,s,v = hsv_img[:,:,0],hsv_img[:,:,1],hsv_img[:,:,2]
    # use the floodfill to fill the image.
    flood_img = src_image.copy()
    seed_cnt = 0
    for i in range(rand_seed_num):
        rand_index = np.random.choice(rand_seed_num,1,replace=False)
        row,col = row_of_points[rand_index],col_of_points[rand_index]
        # To limit the background color of the seed.
        if (((h[row,col]>26)&(h[row,col]<34))|((h[row,col]>100)&(h[row,col]<124)))&(s[row,col]>70)&(v[row,col]>70):
            cv2.floodFill(src_image, mask_point, (int(col),int(row)), (255, 255, 255), (lower,) * 3, (upper,) * 3, flags)
            cv2.circle(flood_img,center=(int(col),int(row)),radius=2,color=(0,0,255),thickness=2)
            seed_cnt += 1
            if seed_cnt >= valid_seed_num:
                break
    # print the image after floodfill algorithm
    show_seed = np.random.uniform(1,100,1).astype(np.uint16)
    #cv2.imshow('floodfill'+str(show_seed),flood_img)
    #cv2.imshow('flood_mask'+str(show_seed),mask_point)
    newpoints_mask = []
    for row in range(1,height+1):
        for col in range(1,wedith+1):
            if mask_point[row,col] != 0:
                newpoints_mask.append((col-1,row-1))
    rotated = cv2.minAreaRect(np.array(newpoints_mask))
    if vscale(rotated):
        return True,rotated
    else:
        return False,rotated

# split the license plate out for a single picture
def carPlateLocation(orig,pred):
    carPlate_list = []
    temp1_orig = orig.copy() 
    temp2_orig = orig.copy() 
    contours,heriachy = cv2.findContours(pred,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for i,contour in enumerate(contours):
        cv2.drawContours(temp1_orig, contours, i, (0, 255, 255), 2)
        # get the smallest in circle rectangle，return the value of rotate_rect
        rotate = cv2.minAreaRect(contour)
        # based on rectanle area and lenght ratio to find there is a license plate or not
        if vscale(rotate):
            ret,rotate2 = verify_color(rotate,temp2_orig)
            if ret == False:
                continue
            # License plate position correction
            car_plate = image_Transform(rotate2, temp2_orig)
            car_plate = cv2.resize(car_plate,(plate_width,plate_height)) #resize the car plate for CNN input to use
            box = cv2.boxPoints(rotate2)
            for k in range(4):
                n1,n2 = k%4,(k+1)%4
                cv2.line(temp1_orig,(box[n1][0],box[n1][1]),(box[n2][0],box[n2][1]),(255,0,0),2)
            #cv2.imshow('opencv_' + str(i), car_plate)
            carPlate_list.append(car_plate)

    #cv2.imshow('contour', temp1_orig_img)
    return carPlate_list

# cut the character out
def split_character(plate):
    char_addr_list = []
    area_left,rightarea,leftCharacter,char_right= 0,0,0,0
    weidith1 = plate.shape[1]
    def getColSum(img,col):
        sum = 0
        for i in range(img.shape[0]):
            sum += round(img[i,col]/255)
        return sum;
    sum = 0
    for col in range(weidith1):
        sum += getColSum(plate,col)
    limit_col = 0 #round(0.5*sum/img_w)
    # limit the ratio of height and width of the license plate.
    charWid_limit = [round(weidith1/12),round(weidith1/5)]
    is_char_flag = False
    # To Canny the lincense plate
    for i in range(weidith1):
        Value_col = getColSum(plate,i)
        if Value_col > limit_col:
            if is_char_flag == False:
                rightarea = round((i+char_right)/2)
                area_width = rightarea-area_left
                char_width = char_right-leftCharacter
                if (area_width>charWid_limit[0]) and (area_width<charWid_limit[1]):
                    char_addr_list.append((area_left,rightarea,char_width))
                leftCharacter = i
                area_left = round((leftCharacter+char_right) / 2)
                is_char_flag = True
        else:
            if is_char_flag == True:
                char_right = i-1
                is_char_flag = False
    if rightarea < leftCharacter:
        rightarea,char_right = weidith1,weidith1
        area_width = rightarea - area_left
        char_width = char_right - leftCharacter
        if (area_width > charWid_limit[0]) and (area_width < charWid_limit[1]):
            char_addr_list.append((area_left, rightarea, char_width))
    return char_addr_list


# use the distance between white area to split each area then spilt the character out.
def character_split(car_plate):
    heigth,weigth = car_plate.shape[:2]
    h_proj_list = []
    h_temp_len,v_temp_len = 0,0
    h_startIndex,h_end_index = 0,0
    h_proj_limit = [0.2,0.8]
    char_imgs = []
    h_count = [0 for i in range(heigth)]
    for row in range(heigth):
        temp_cnt = 0
        for col in range(weigth):
            if car_plate[row,col] == 255:
                temp_cnt += 1
        h_count[row] = temp_cnt
        if temp_cnt/weigth<h_proj_limit[0] or temp_cnt/weigth>h_proj_limit[1]:
            if h_temp_len != 0:
                h_end_index = row-1
                h_proj_list.append((h_startIndex,h_end_index))
                h_temp_len = 0
            continue
        if temp_cnt > 0:
            if h_temp_len == 0:
                h_startIndex = row
                h_temp_len = 1
            else:
                h_temp_len += 1
        else:
            if h_temp_len > 0:
                h_end_index = row-1
                h_proj_list.append((h_startIndex,h_end_index))
                h_temp_len = 0
    if h_temp_len != 0:
        h_end_index = heigth-1
        h_proj_list.append((h_startIndex, h_end_index))
    h_maxIndex,h_maxHeight = 0,0
    for i,(start,end) in enumerate(h_proj_list):
        if h_maxHeight < (end-start):
            h_maxHeight = (end-start)
            h_maxIndex = i
    if h_maxHeight/heigth < 0.5:
        return char_imgs
    chars_top,chars_bottom = h_proj_list[h_maxIndex][0],h_proj_list[h_maxIndex][1]
    plates = car_plate[chars_top:chars_bottom+1,:]
    cv2.imwrite('./Dataset/opencv/car.jpg',car_plate)
    cv2.imwrite('./Dataset/opencv/plate.jpg', plates)
    char_addr_list = split_character(plates)
    for i,addr in enumerate(char_addr_list):
        char_img = car_plate[chars_top:chars_bottom+1,addr[0]:addr[1]]
        char_img = cv2.resize(char_img,(char_width,char_height))
        char_imgs.append(char_img)
    return char_imgs

# Separate the character out
def extract_char(car_plate):
    gray_plate = cv2.cvtColor(car_plate,cv2.COLOR_BGR2GRAY)
    ret,binary_plate = cv2.threshold(gray_plate,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    char_img_list = character_split(binary_plate)
    return char_img_list

# use the trained CNN license plate recognition model to check there is a license plate or not
def cnncarPlate(plate_list,paths):
    if len(plate_list) == 0:
        return False,plate_list
    graphh = tf.Graph()
    session1 = tf.Session(graph=graphh)
    with session1.as_default():
        with session1.graph.as_default():
            model_pos = os.path.dirname(paths)
            savers = tf.train.import_meta_graph(paths)
            savers.restore(session1, tf.train.latest_checkpoint(model_pos))
            graph = tf.get_default_graph()
            net1_x_position = graph.get_tensor_by_name('x_place:0')
            net1_keep_position = graph.get_tensor_by_name('keep_place:0')
            net1_output = graph.get_tensor_by_name('out_put:0')

            input_x = np.array(plate_list)
            net_out = tf.nn.softmax(net1_output)
            pred = tf.argmax(net_out,1) #predit the results
            prob = tf.reduce_max(net_out,reduction_indices=[1]) #result prob
            pred_list,prob_list = session1.run([pred,prob],feed_dict={net1_x_position:input_x,net1_keep_position:1.0})
            # choose the highest prob license plate
            result_index,result_prob = -1,0.
            for i,pred in enumerate(pred_list):
                if pred==1 and prob_list[i]>result_prob:
                    result_index,result_prob = i,prob_list[i]
            # boolen variable will use to main program for check the license plate exist or not
            if result_index == -1:
                return False,plate_list[0]
            else:
                return True,plate_list[result_index]
            
# use the trained CNN character recognition model to recognize characters numbers, characters, chinese character
# in the table
def cnnchar(img_list,paths):
    graphhh = tf.Graph()
    session2 = tf.Session(graph=graphhh)
    text_lists = []

    if len(img_list) == 0:
        return text_lists
    with session2.as_default():
        with session2.graph.as_default():
            model_pos = os.path.dirname(paths)
            savers = tf.train.import_meta_graph(paths)
            savers.restore(session2, tf.train.latest_checkpoint(model_pos))
            graph = tf.get_default_graph()
            net2_x_position = graph.get_tensor_by_name('x_place:0')
            net2_keep_position = graph.get_tensor_by_name('keep_place:0')
            net2_output = graph.get_tensor_by_name('out_put:0')

            data = np.array(img_list)
            # numbers, characters, chinese character，total 67 characters
            # find the highest prob. results from the table match labels
            net_outs = tf.nn.softmax(net2_output)
            pred = tf.argmax(net_outs,1)
            my_pred= session2.run(pred, feed_dict={net2_x_position: data, net2_keep_position: 1.0})

            for i in my_pred:
                text_lists.append(character[i])
            return text_lists

# Main Program

In [11]:
cur_pos = sys.path[0]
plate_width,plate_height = 136,36
char_width,char_height = 20,20
plate_model = os.path.join(cur_pos, './Dataset/models/plate/model.ckpt-510.meta')
char_model = os.path.join(cur_pos,'./Dataset/models/char/model.ckpt-510.meta')
img = cv2.imread('./Dataset/test_images/1.jpg')

#preprocessing
pred_img = preprocessing(img)

# car plate location
plate_list = carPlateLocation(img,pred_img)

# CNN car plate recognition
bools,car_plate = cnncarPlate(plate_list,plate_model)
8
if bools == False:
    print("No License Plates Check Your Image, Try Again")
    sys.exit(-1)
#cv2.imshow('cnn_plate',car_plate)

# character extract
char_image_list = extract_char(car_plate)

# CNN Character Recognition
char_text = cnnchar(char_image_list,char_model)
print(char_text)

INFO:tensorflow:Restoring parameters from /Users/evan/MS/Ece697/bishe /1/./Dataset/models/plate/model.ckpt-510
INFO:tensorflow:Restoring parameters from /Users/evan/MS/Ece697/bishe /1/./Dataset/models/char/model.ckpt-510
['浙', 'A', 'L', 'P', '5', '2', '0']
